# Parse real time hydrometric data
Parse real time data from https://wateroffice.ec.gc.ca/mainmenu/real_time_data_index_e.html .

[beautifulsoup](https://beautiful-soup-4.readthedocs.io/) is a popular python library for parsing html. If you found yourself needing to scrape javascript or programatically clicking on buttons, try [selenium](https://selenium-python.readthedocs.io/) instead. 

In this example, we directly download the data in CSV format to avoid needing to scrape html.

 ## Install dependencies
 [Helpful notes](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/) on best practices for installing dependencies with jupyter notebooks.

In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4 httpx pandas

You should consider upgrading via the '/home/jthetzel/.cache/pypoetry/virtualenvs/notebooks-4iuYC0qS-py3.10/bin/python -m pip install --upgrade pip' command.


## Import dependencies

In [2]:
import httpx
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

## Function to download data
If we have time later, we will discuss more detail on how this function was developed.

In [5]:
def get_hydrometric_data(station_id: str, date_start: str, date_end: str) -> pd.DataFrame:
    """Fetch hydrometic data from wateroffice.ec.gc.ca and return pandas data frame."""
    
    # Uri to create a session with wateroffice.ec.gc.ca
    uri_station = f"https://wateroffice.ec.gc.ca/report/real_time_e.html?stn={station_id}&mode=Table&startDate={date_start}&endDate={date_end}"
    
    # Uri to accept the terms and conditions, which updates our session that terms are accepted
    uri_login = "https://wateroffice.ec.gc.ca/disclaimer_e.html"
    
    # Once our session is updated with accepted terms, we can use this uri to downoad the data as a csv zip archive
    uri_csv = f"https://wateroffice.ec.gc.ca/download/report_e.html?dt=47&df=csv&ext=zip&stn={station_id}&mode=Table&startDate={date_start}&endDate={date_end}"


    # Get the uri_station to create our session
    with httpx.Client(timeout=10, follow_redirects=True) as client:
        response_station = client.get(uri_station)
        response_station.raise_for_status

    # Get the session id from the response cookie. We'll use this for future http requests.
    PHPSESSID = response_station.cookies["PHPSESSID"]
    cookies = httpx.Cookies()
    cookies.set("PHPSESSID", PHPSESSID, domain="wateroffice.ec.gc.ca")


    # Accept terms and download data. We'll share our cookies so both requests use the same session id.
    with httpx.Client(timeout=10, follow_redirects=True, cookies=cookies) as client:
        headers = {"referrer": uri_station}
        data = {"disclaimer_action": "I+Agree"}

        # Accept terms
        response_login = client.post(uri_login, data=data, headers=headers)
        response_login.raise_for_status()

        # Download data
        response_csv = client.get(uri_csv)
        response_csv.raise_for_status()

    # The data are returned as a stream of bytes representing a csv zip archive file. Open the zip archive in memory.
    zip_file = ZipFile(BytesIO(response_csv.content))
    data_file = zip_file.filelist[0]

    # Read the csv file as a stream of bytes into a pandas data frame, skipping non-data rows
    data_frame = pd.read_csv(BytesIO(zip_file.read(data_file.filename)), skiprows=9)
    
    # Return the data frame
    return data_frame

## Download data
Download data from wateroffice.ec.gc.ca, specifying the station_id, date_start, and date_end

In [6]:
station_id = "10JC003"
date_start = "2022-11-07"
date_end = "2022-11-14"

data_frame = get_hydrometric_data(station_id, date_start, date_end)
data_frame

,Date (MST),Parameter,Value (m³/s),Approval,Qualifier
0,2022-11-07 00:00:00,47,561,PRELIMINARY,UNSPECIFIED
1,2022-11-07 00:05:00,47,560,PRELIMINARY,UNSPECIFIED
2,2022-11-07 00:10:00,47,558,PRELIMINARY,UNSPECIFIED
3,2022-11-07 00:15:00,47,558,PRELIMINARY,UNSPECIFIED
4,2022-11-07 00:20:00,47,559,PRELIMINARY,UNSPECIFIED
...,...,...,...,...,...
2299,2022-11-14 23:35:00,47,518,PRELIMINARY,UNSPECIFIED
2300,2022-11-14 23:40:00,47,515,PRELIMINARY,UNSPECIFIED
2301,2022-11-14 23:45:00,47,513,PRELIMINARY,UNSPECIFIED
2302,2022-11-14 23:50:00,47,515,PRELIMINARY,UNSPECIFIED


## Done!
We now have hydrometric data in a pandas data frame. Not sure how robust `get_hydrometric_data` is to modifying the station_id, date_start, or date_end arguments, but hopefully if we get the patterns right, this should be automatable.

Below are some notes to self that we might make into a more in depth tutorial of using http responses, cookies, sessions, and byte streams to access data. If we ever have time. Ignore for now.

## Login to the website

In [ ]:
url = "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"

After playing around with attempting to download the data directly in csv form, it seems the gc.ca website expects a PHP session token. Without it, we get redirected to the login page. By opening the browser developer tools (ctrl-shift-i in chrome) and clicking the `network` tab, we can see the actuall http requests made by the website. If we then click the `agree` button in the login page, we see a POST request to `disclaimer_e.html` in the network tab. Right-clicking this, we can select `copy as fetch` which copies into our clipboard the following:

```
fetch("https://wateroffice.ec.gc.ca/disclaimer_e.html", {
  "headers": {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "content-type": "application/x-www-form-urlencoded",
    "sec-ch-ua": "\"Google Chrome\";v=\"105\", \"Not)A;Brand\";v=\"8\", \"Chromium\";v=\"105\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Linux\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1"
  },
  "referrer": "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": "disclaimer_action=I+Agree",
  "method": "POST",
  "mode": "cors",
  "credentials": "include"
});
```

The above is the exact same http post request made by the browser, but translated into javascript (`fetch` is a built-in javascript function for making http requests). We can now translate that into a python function to make a similar request from python.

In [11]:
with httpx.Client(timeout=10, follow_redirects=True) as client:
    headers = {}
    referrer = "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"
    body = "disclaimer_action=I+Agree"
    response = client.post("https://wateroffice.ec.gc.ca/disclaimer_e.html", data=body)
    
response    

<Response [200 OK]>

THe `200` indicates a success! Other http codes include `300` codes (redirect to another site), `400` codes (malformed request or permission error), or `500` codes (something went wrong on the server). We can inspect the `text` property, which is the html file returned after logging in.

Here are the other properties and methods available to the `response` object.

In [15]:
dir(response)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_decoder',
 '_elapsed',
 '_encoding',
 '_get_content_decoder',
 '_num_bytes_downloaded',
 '_prepare',
 '_request',
 '_text',
 'aclose',
 'aiter_bytes',
 'aiter_lines',
 'aiter_raw',
 'aiter_text',
 'apparent_encoding',
 'aread',
 'charset_encoding',
 'close',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'extensions',
 'has_redirect_location',
 'headers',
 'history',
 'http_version',
 'is_client_error',
 'is_closed',
 'is_error',
 'is_informational',
 'is_redirect',
 'is_server_error',
 'is_stream_consumed',
 'is_success',
 'iter_bytes',
 'iter_lines',
 'iter_raw',
 'ite

In [31]:
with httpx.Client(timeout=10, follow_redirects=True) as client:
    referrer = "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"
    headers = {"referrer": referrer}
    data = {"disclaimer_action": "I+Agree"}
    response = client.post("https://wateroffice.ec.gc.ca/disclaimer_e.html", data=body, headers=headers)
    
response  

<Response [200 OK]>

In [50]:
uri = "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"

In [51]:
with httpx.Client(timeout=10, follow_redirects=True) as client:
    response = client.get(uri)
    response.raise_for_status
    
response 

<Response [200 OK]>

In [52]:
response.cookies

<Cookies[<Cookie PHPSESSID=189d6da20ece6a5acaf1d82a883852d9 for wateroffice.ec.gc.ca />]>

In [53]:
PHPSESSID = response.cookies["PHPSESSID"]
PHPSESSID

'189d6da20ece6a5acaf1d82a883852d9'

In [54]:
cookies = httpx.Cookies()
cookies.set("PHPSESSID", PHPSESSID, domain="wateroffice.ec.gc.ca")

In [130]:
zip_file = ZipFile(BytesIO(response_csv.content))
zip_file

<zipfile.ZipFile file=<_io.BytesIO object at 0x7f8d86ed2c00> mode='r'>

In [131]:
data_file = zip_file.filelist[0]
data_file

<ZipInfo filename='10JC003_QR_20221118T1930.csv' compress_type=deflate filemode='-rw-rw-rw-' file_size=120174 compress_size=8962>

In [138]:
pd.read_csv(BytesIO(zip_file.read(data_file.filename)), skiprows=9)

,Date (MST),Parameter,Value (m³/s),Approval,Qualifier
0,2022-11-07 00:00:00,47,561,PRELIMINARY,UNSPECIFIED
1,2022-11-07 00:05:00,47,560,PRELIMINARY,UNSPECIFIED
2,2022-11-07 00:10:00,47,558,PRELIMINARY,UNSPECIFIED
3,2022-11-07 00:15:00,47,558,PRELIMINARY,UNSPECIFIED
4,2022-11-07 00:20:00,47,559,PRELIMINARY,UNSPECIFIED
...,...,...,...,...,...
2299,2022-11-14 23:35:00,47,518,PRELIMINARY,UNSPECIFIED
2300,2022-11-14 23:40:00,47,515,PRELIMINARY,UNSPECIFIED
2301,2022-11-14 23:45:00,47,513,PRELIMINARY,UNSPECIFIED
2302,2022-11-14 23:50:00,47,515,PRELIMINARY,UNSPECIFIED


In [ ]:


with httpx.Client(timeout=10, follow_redirects=True, cookies=cookies) as client:
    referrer = "https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"
    login_uri = "https://wateroffice.ec.gc.ca/disclaimer_e.html"
    # download_uri = "https://wateroffice.ec.gc.ca/download/report_e.html?dt=47&df=txt&ext=zip&stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"
    download_uri = "https://wateroffice.ec.gc.ca/download/report_e.html?dt=47&df=txt&stn=10JC003&mode=Table&startDate=2022-11-07&endDate=2022-11-14"
    headers = {"referrer": referrer}
    data = {"disclaimer_action": "I+Agree"}
    response = client.post(login_uri, data=body, headers=headers)
    
    response.raise_for_status()
    
    download_response = client.get(download_uri)
    
download_response

In [71]:
download_response.encoding

'utf-8'

In [68]:
download_response.text

'PK\x03\x04\x14\x00\x00\x00\x08\x00��rU��\t�\x02#\x00\x00n�\x01\x00\x1c\x00\x00\x0010JC003_QR_20221118T1807.txt��ͮ\x1e�q��\x14�{8��Z\x08�����N��D��8�l \x08�`$�f`��!-�וe\x16\x01rC���>_��;\x01��)@� ��W�55�W�N��\x7f��/_�\U00053def�}��͋�/\x1e>yx�\x7f���_�}x�����\x0f�\u07bcz�݇\x1f|���??���\x1f�><��W������wo\x1f>{�\u074bo^�$���?���\x0f>�ˏ}�������<������W_?|���\x7f����o�|��W\x0f\x7f���/\x1f���Ͽ|x�_��~��ݿ~��ߟ\x7f���\x1f~Ҷ��l���\x0f��˯���\x06n\x7f��#�K���_������o�u����\x1e�\x7f���]�^>���\x1f���G��z��\x1f^<����O�\x7f|���??�y�ݛwm����\x1f^~��GϾ��\x7f��7�[�?���/~����~��?���/���w\x7f�����O�����\x0f/���\x1f��⏯~����\x1f���\'�}�\x1d\x0f��ӿ�����۳_}���_��_~��?=��/\x7f���}�/�\x7f�\x7f����\\��h\'��$g�n紙�L/��w;�ˌ�������2Y��w;�1m�7�mm���.�\nv�x;\\&���\x1aovۖxk6s����\x12o��G�N\x16�$�[���\x0c��X�m����%��f\nvz�o\x1d�?Q\x18�b�7��\x12o>���5�N�I:\x07�%�\x00��mQ�\x13x}Зx��f�vv����>l�n\']�\x1eo{���m\x1b����LR_�B��%���D�?��`,�v�����a��\x1eo��۱}���i3�o��۰���%�|�n�p�%ަ�$�u���L�-��Q�O\x16����ٗxs��}���2<��5��ex��k��L�q��

In [67]:
dir(download_response)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_decoder',
 '_elapsed',
 '_encoding',
 '_get_content_decoder',
 '_num_bytes_downloaded',
 '_prepare',
 '_request',
 '_text',
 'aclose',
 'aiter_bytes',
 'aiter_lines',
 'aiter_raw',
 'aiter_text',
 'apparent_encoding',
 'aread',
 'charset_encoding',
 'close',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'extensions',
 'has_redirect_location',
 'headers',
 'history',
 'http_version',
 'is_client_error',
 'is_closed',
 'is_error',
 'is_informational',
 'is_redirect',
 'is_server_error',
 'is_stream_consumed',
 'is_success',
 'iter_bytes',
 'iter_lines',
 'iter_raw',
 'ite